# Imports

In [1]:
import torch
import dqc
import dqc.xc
import dqc.utils
from importlib import reload 
import DQCAdapter as evg
from pyscf import gto, dft

## Init for DQC

In [14]:
class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a_par, p_par):
        super().__init__()
        self.a_par = a_par
        self.p_par = p_par
        self.number_of_parameters = 2

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a_par * rho ** self.p_par
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p_par
            elif number_of_parameter == 1: # parameter p
                return self.a_par * torch.log(rho) * rho ** self.p_par

## Custom xc for PySCF

In [15]:
def custom_eval_xc(a, p):
    def eval_xc(xc_code, rho, *args, **kwargs):
        exc = a * rho ** (p - 1.0)
        vrho = a * p * rho ** (p - 1.0)
        vgamma = None
        vlapl = None
        vtau = None
        vxc = (vrho, vgamma, vlapl, vtau)
        fxc = None  # 2nd order functional derivative
        kxc = None  # 3rd order functional derivative
        return exc, vxc, fxc, kxc
    return eval_xc

## Systems

In [16]:
water = """
O        0.000000000      0.000000000      0.000000000;
H        0.000000000      1.434938863      1.126357947;
H        0.000000000     -1.434938863      1.126357947
"""

h2_system = "H 0 0 0; H 1.4 0 0"

n2_system = "N 0 0 0; N 2.07 0 0"

ammonia = "N 0.0 0.0 0.0; H 0.0 -1.772 -0.721; H 1.535 0.886 -0.721; H -1.535 0.886 -0.721"

## Methods


In [30]:
def PySCF_calculation(system, a, p, verbose=0):
    xc = custom_eval_xc(a,p)
    mol = gto.M(atom = system, basis="6-31G", unit = 'Bohr')
    calc = dft.RKS(mol)
    calc = calc.define_xc_(xc, 'LDA')
    calc.verbose = verbose
    ene = calc.kernel()
    return ene

def DQC_calculation_tensors(system, a, p):
    atomzs, atomposs = dqc.parse_moldesc(system)
    mol = dqc.Mol((atomzs, atomposs), basis="6-31G")
    myxc = MyLDAX(a, p)
    qc = dqc.KS(mol, xc=myxc).run()
    return qc

def DQC_calculation(system, a_value, p_value):
    a = torch.nn.Parameter(torch.tensor(a_value, dtype=torch.double))
    p = torch.nn.Parameter(torch.tensor(p_value, dtype=torch.double))
    return DQC_calculation_tensors(system, a, p)

def finite_difference(system, a_value, p_value, coeff=0.000001):
    y = DQC_calculation(system, a_value, p_value).energy().detach()
    print("ENE DQC: ", y)
    da = a_value * coeff
    dp = p_value * coeff
    y_plus_dy_a = DQC_calculation(system, a_value+da, p_value)
    y_plus_dy_p = DQC_calculation(system, a_value, p_value+dp)
    dy_wrt_da = (y_plus_dy_a.energy().detach() - y) / da
    dy_wrt_dp = (y_plus_dy_p.energy().detach() - y) / dp
    return torch.tensor((dy_wrt_da, dy_wrt_dp))

def finite_difference_pyscf(system, a, p, coeff=0.000001, verbose = 0):
    y = PySCF_calculation(system, a, p, verbose)
    print("ENE PYSCF: ", y)
    da = a * coeff
    dp = p * coeff
    y_plus_dy_a = PySCF_calculation(system, a+da, p)
    y_plus_dy_p = PySCF_calculation(system, a, p+dp)
    dy_wrt_da = (y_plus_dy_a - y) / da
    dy_wrt_dp = (y_plus_dy_p - y) / dp
    return torch.tensor((dy_wrt_da, dy_wrt_dp))


def reverse_mode(system, a_value, p_value):
    a = torch.nn.Parameter(torch.tensor(a_value, dtype=torch.double))
    p = torch.nn.Parameter(torch.tensor(p_value, dtype=torch.double))
    ene = DQC_calculation_tensors(system, a, p).energy()
    print("ENE REVERSE: ", ene.detach())
    grad_a, grad_p = torch.autograd.grad(ene, (a, p))
    return torch.tensor((grad_a, grad_p))

### Params a = 1.0, p = 2.0

In [23]:
print("REVERSE: ", reverse_mode(water, 1.0, 2.0))
print("FIN DQC: ", finite_difference(water, 1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(water, 1.0, 2.0))

ENE REVERSE:  tensor(-38.2050, dtype=torch.float64)
REVERSE:  tensor([ 9.9156, 19.0426], dtype=torch.float64)
ENE DQC:  tensor(-38.2050, dtype=torch.float64)
FIN DQC:  tensor([ 9.9156, 19.0426], dtype=torch.float64)
ENE PYSCF:  -38.20496753433136
FIN PYSCF:  tensor([ 9.9156, 19.0425], dtype=torch.float64)


In [25]:
print("REVERSE: ", reverse_mode(h2_system, 1.0, 2.0))
print("FIN DQC: ", finite_difference(h2_system, 1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(h2_system, 1.0, 2.0))

ENE REVERSE:  tensor(-0.4216, dtype=torch.float64)
REVERSE:  tensor([ 0.0885, -0.2367], dtype=torch.float64)
ENE DQC:  tensor(-0.4216, dtype=torch.float64)
FIN DQC:  tensor([ 0.0885, -0.2367], dtype=torch.float64)
ENE PYSCF:  -0.4216044717484252
FIN PYSCF:  tensor([ 0.0885, -0.2367], dtype=torch.float64)


In [31]:
print("REVERSE: ", reverse_mode(n2_system, 1.0, 2.0))
print("FIN DQC: ", finite_difference(n2_system, 1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(n2_system, 1.0, 2.0, verbose=4))

ENE REVERSE:  tensor(-54.3595, dtype=torch.float64)
REVERSE:  tensor([14.7695, 26.0360], dtype=torch.float64)
ENE DQC:  tensor(-54.3595, dtype=torch.float64)
FIN DQC:  tensor([-14434.5899,  -9749.0031], dtype=torch.float64)


******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/sabina/noa/docs/qc/tmpuymz08du
max_memory 4000 MB (current use 386 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = LDA,VWN
    P. A. M. Dirac, Math. Proc. Cambridge Philos. Soc. 26, 376 (1930)
    F. Bloch, Z. Phys. 57, 545 (1929)
    S. H. Vosko, L. Wilk, and M. Nusair, Can. J. Phys. 58, 1200 (1980)
small_rho_cutoff = 1e-07
Se

In [32]:
print("REVERSE: ", reverse_mode(ammonia, 1.0, 2.0))
print("FIN DQC: ", finite_difference(ammonia, 1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(ammonia, 1.0, 2.0))

ENE REVERSE:  tensor(-28.1712, dtype=torch.float64)
REVERSE:  tensor([ 7.4649, 12.7271], dtype=torch.float64)
ENE DQC:  tensor(-28.1712, dtype=torch.float64)
FIN DQC:  tensor([ 7.4649, 12.7271], dtype=torch.float64)
ENE PYSCF:  -28.171179569131418
FIN PYSCF:  tensor([ 7.4649, 12.7270], dtype=torch.float64)


### Params a = 1.0, p = 3.0

In [33]:
print("REVERSE: ", reverse_mode(water, 1.0, 3.0))
print("FIN DQC: ", finite_difference(water, 1.0, 3.0))
print("FIN PYSCF: ", finite_difference_pyscf(water, 1.0, 3.0, verbose=4))

/home/sabina/miniconda3/lib/python3.8/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=1.543e+03, |f|=1.529e+03 at iter 50
  warnings.warn(ConvergenceWarning(msg))


ENE REVERSE:  tensor(-23.4711, dtype=torch.float64)
REVERSE:  tensor([ 0.7208, -0.5884], dtype=torch.float64)
ENE DQC:  tensor(-23.4711, dtype=torch.float64)
FIN DQC:  tensor([2.4030, 3.8163], dtype=torch.float64)


******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/sabina/noa/docs/qc/tmpeb7grllk
max_memory 4000 MB (current use 382 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = LDA,VWN
    P. A. M. Dirac, Math. Proc. Cambridge Philos. Soc. 26, 376 (1930)
    F. Bloch, Z. Phys. 57, 545 (1929)
    S. H. Vosko, L. Wilk, and M. Nusair, Can. J. Phys. 58, 1200 (1980)
small_rho_cutoff = 1e-07
Set gradient

In [34]:
print("REVERSE: ", reverse_mode(h2_system, 1.0, 3.0))
print("FIN DQC: ", finite_difference(h2_system, 1.0, 3.0))
print("FIN PYSCF: ", finite_difference_pyscf(h2_system, 1.0, 3.0, verbose=4))

ENE REVERSE:  tensor(-0.5084, dtype=torch.float64)
REVERSE:  tensor([ 0.0106, -0.0225], dtype=torch.float64)
ENE DQC:  tensor(-0.5084, dtype=torch.float64)
FIN DQC:  tensor([ 0.0106, -0.0225], dtype=torch.float64)


******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/sabina/noa/docs/qc/tmpkrx6ef9v
max_memory 4000 MB (current use 382 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = LDA,VWN
    P. A. M. Dirac, Math. Proc. Cambridge Philos. Soc. 26, 376 (1930)
    F. Bloch, Z. Phys. 57, 545 (1929)
    S. H. Vosko, L. Wilk, and M. Nusair, Can. J. Phys. 58, 1200 (1980)
small_rho_cutoff = 1e-07
Set gradient

In [35]:
print("REVERSE: ", reverse_mode(n2_system, 1.0, 3.0))
print("FIN DQC: ", finite_difference(n2_system, 1.0, 3.0))
print("FIN PYSCF: ", finite_difference_pyscf(n2_system, 1.0, 3.0, verbose=4))

/home/sabina/miniconda3/lib/python3.8/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=9.276e+02, |f|=9.204e+02 at iter 50
  warnings.warn(ConvergenceWarning(msg))


ENE REVERSE:  tensor(-33.5378, dtype=torch.float64)
REVERSE:  tensor([ 1.4839, -1.2046], dtype=torch.float64)
ENE DQC:  tensor(-33.5378, dtype=torch.float64)
FIN DQC:  tensor([4.8555, 9.3064], dtype=torch.float64)


******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/sabina/noa/docs/qc/tmp1yg3osp6
max_memory 4000 MB (current use 383 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = LDA,VWN
    P. A. M. Dirac, Math. Proc. Cambridge Philos. Soc. 26, 376 (1930)
    F. Bloch, Z. Phys. 57, 545 (1929)
    S. H. Vosko, L. Wilk, and M. Nusair, Can. J. Phys. 58, 1200 (1980)
small_rho_cutoff = 1e-07
Set gradient

In [36]:
print("REVERSE: ", reverse_mode(ammonia, 1.0, 3.0))
print("FIN DQC: ", finite_difference(ammonia, 1.0, 3.0))
print("FIN PYSCF: ", finite_difference_pyscf(ammonia, 1.0, 3.0, verbose=4))

/home/sabina/miniconda3/lib/python3.8/site-packages/xitorch/_impls/optimize/root/rootsolver.py:163: ConvergenceWarning: The rootfinder does not converge after 50 iterations. Best |dx|=6.646e+02, |f|=6.592e+02 at iter 50
  warnings.warn(ConvergenceWarning(msg))


ENE REVERSE:  tensor(-17.3388, dtype=torch.float64)
REVERSE:  tensor([ 0.5239, -0.5451], dtype=torch.float64)
ENE DQC:  tensor(-17.3388, dtype=torch.float64)
FIN DQC:  tensor([2.0833, 3.6417], dtype=torch.float64)


******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/sabina/noa/docs/qc/tmpcjayijsj
max_memory 4000 MB (current use 440 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = LDA,VWN
    P. A. M. Dirac, Math. Proc. Cambridge Philos. Soc. 26, 376 (1930)
    F. Bloch, Z. Phys. 57, 545 (1929)
    S. H. Vosko, L. Wilk, and M. Nusair, Can. J. Phys. 58, 1200 (1980)
small_rho_cutoff = 1e-07
Set gradient

### Params a = -1.0, p = 2.0

In [42]:
print("REVERSE: ", reverse_mode(water, -1.0, 2.0))
print("FIN DQC: ", finite_difference(water, -1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(water, -1.0, 2.0, verbose = 4))

ENE REVERSE:  tensor(-155.4981, dtype=torch.float64)
REVERSE:  tensor([  96.9847, -369.9213], dtype=torch.float64)
ENE DQC:  tensor(-155.4981, dtype=torch.float64)
FIN DQC:  tensor([  96.9847, -369.9230], dtype=torch.float64)


******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/sabina/noa/docs/qc/tmplpuzn5hf
max_memory 4000 MB (current use 385 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = LDA,VWN
    P. A. M. Dirac, Math. Proc. Cambridge Philos. Soc. 26, 376 (1930)
    F. Bloch, Z. Phys. 57, 545 (1929)
    S. H. Vosko, L. Wilk, and M. Nusair, Can. J. Phys. 58, 1200 (1980)
small_rho_cutoff = 1e-07


In [38]:
print("REVERSE: ", reverse_mode(h2_system, -1.0, 2.0))
print("FIN DQC: ", finite_difference(h2_system, -1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(h2_system, -1.0, 2.0, verbose = 4))

ENE REVERSE:  tensor(-0.6463, dtype=torch.float64)
REVERSE:  tensor([0.1487, 0.3173], dtype=torch.float64)
ENE DQC:  tensor(-0.6463, dtype=torch.float64)
FIN DQC:  tensor([0.1487, 0.3173], dtype=torch.float64)


******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/sabina/noa/docs/qc/tmp5rcu2uz2
max_memory 4000 MB (current use 381 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = LDA,VWN
    P. A. M. Dirac, Math. Proc. Cambridge Philos. Soc. 26, 376 (1930)
    F. Bloch, Z. Phys. 57, 545 (1929)
    S. H. Vosko, L. Wilk, and M. Nusair, Can. J. Phys. 58, 1200 (1980)
small_rho_cutoff = 1e-07
Set gradient con

In [39]:
print("REVERSE: ", reverse_mode(n2_system, -1.0, 2.0))
print("FIN DQC: ", finite_difference(n2_system, -1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(n2_system, -1.0, 2.0, verbose = 4))

ENE REVERSE:  tensor(-206.3187, dtype=torch.float64)
REVERSE:  tensor([ 113.7847, -413.4969], dtype=torch.float64)
ENE DQC:  tensor(-206.3187, dtype=torch.float64)
FIN DQC:  tensor([ 113.7847, -413.4986], dtype=torch.float64)


******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/sabina/noa/docs/qc/tmpddhg89m0
max_memory 4000 MB (current use 374 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = LDA,VWN
    P. A. M. Dirac, Math. Proc. Cambridge Philos. Soc. 26, 376 (1930)
    F. Bloch, Z. Phys. 57, 545 (1929)
    S. H. Vosko, L. Wilk, and M. Nusair, Can. J. Phys. 58, 1200 (1980)
small_rho_cutoff = 1e-07


In [40]:
print("REVERSE: ", reverse_mode(ammonia, -1.0, 2.0))
print("FIN DQC: ", finite_difference(ammonia, -1.0, 2.0))
print("FIN PYSCF: ", finite_difference_pyscf(ammonia, -1.0, 2.0, verbose = 4))

ENE REVERSE:  tensor(-104.3939, dtype=torch.float64)
REVERSE:  tensor([  58.7489, -208.1992], dtype=torch.float64)
ENE DQC:  tensor(-104.3939, dtype=torch.float64)
FIN DQC:  tensor([  58.7489, -208.2000], dtype=torch.float64)


******** <class 'pyscf.dft.rks.RKS'> ********
method = RKS-RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/sabina/noa/docs/qc/tmpdkcpk4m0
max_memory 4000 MB (current use 419 MB)
XC library pyscf.dft.libxc version 5.1.7
    S. Lehtola, C. Steigemann, M. J. Oliveira, and M. A. Marques, SoftwareX 7, 1 (2018)
XC functionals = LDA,VWN
    P. A. M. Dirac, Math. Proc. Cambridge Philos. Soc. 26, 376 (1930)
    F. Bloch, Z. Phys. 57, 545 (1929)
    S. H. Vosko, L. Wilk, and M. Nusair, Can. J. Phys. 58, 1200 (1980)
small_rho_cutoff = 1e-07
